# Null Hypothesis Significance Testing

In [ ]:
import numpy as np
import scipy as sp
import scipy.stats as stats
import matplotlib.pyplot as plt

plt.style.use('ggplot')

## Central Limit Theorem Based Hypothesis Tests

Many of the most common hypothesis tests are based on the central limit theorem.  Our jumping off point are the simple binomial tests we studied earlier in the week.  By applying the central limit theorem, and occasionally a little ingenuity, we can use the same ideas in more general situations.

### The One Sample Approximate Test of Population Proportion

Earlier in the week we had an example where Matt was testing how frequently he can land kickflips.  

Suppose Matt wants to test his hypothesis more rigorously.  Instead of skating for a day, he spends an entire month collecting data.

**Let's say** he attempt 100 kickflips a day, for a total of 3100 kickflips, and he land 2531 of them.


$$ \text{# Kickflips Landed} \approx Binomial(3100, 0.8) $$

In this case, our $N$ is quite large, so it's possible that we have a computer that cannot handle the exact calculations for the binomial distribution (we don't).

Luckily the central limit theorem tells us that a binomial with large $N$ is well approximated by a Normal distribution with the appropriate mean and varaince.

$$ Binomial(3100, 0.8) \approx N(3100 \times 0.8, \sqrt{3100 \times 0.8 \times 0.2}) $$

**Question:** How did I compute the mean and variance of this normal distribution?

In [ ]:
binomial = stats.binom(n=3100, p=0.80)
binomial_mean = 0.8 * 3100
binomial_var = 3100 * 0.8 * 0.2
normal_approx = stats.norm(binomial_mean, np.sqrt(binomial_var))
x = np.linspace(0, 3100, num=3000)

fig, axs = plt.subplots(2, figsize=(16, 6))
bar_sizes = [binomial.pmf(i) for i in range(3101)]
bars = axs[0].bar(range(3101), bar_sizes, color="black", align="center")
axs[0].plot(x, normal_approx.pdf(x), linewidth=3)
axs[0].set_xlim(0, 2600)

bars = axs[1].bar(range(3101), bar_sizes, color="grey", align="center")
axs[1].plot(x, normal_approx.pdf(x), linewidth=3)
axs[1].set_xlim(2400, 2600)

axs[0].set_title("# of Kickflips Landed Under The Null Hypothesis")

The approximation here is very good, so we can replace the exact Binomail distribution with the approximate Normal distribution.

```python
binomial_mean = 0.8 * 3100
binomial_var = 3100 * 0.8 * 0.2
normal_approx = stats.Normal(binomial_mean, np.sqrt(binomial_var))
```

The p-value for this one month experiment is

$$ P(\geq \text{ 2531 Kickflips Landed} \mid \text{Null Hypothesis} ) $$

In [ ]:
p_value = 1 - normal_approx.cdf(2530.5)
print("p-value for one month kickflip experiment: {:2.2f}".format(p_value))

In [ ]:
fig, ax = plt.subplots(1, figsize=(16, 3))

ax.plot(x, normal_approx.pdf(x), linewidth=3)
ax.set_xlim(2400, 2600)
ax.fill_between(x, normal_approx.pdf(x), 
                where=(x >= 2530), color="red", alpha=0.5)
ax.set_title("p-value Reigon")

The month of practice has made Matt much more confident in his skill, he should definately move on to another trick.

The approximate test for a population proportion (the "approximate" because the Binomial distribution is approximated with a Normal) is often called the **z test for a population proportion** because the tables of tail probabilities of normal distributions that poeople would use to look up tail probablities from the normal distribution back in the day were called "z-tables".

It's an amusing fact that crappy undergrad statistics textbooks still print the mandataory z-tables on thier back covers.  In 2017.

**Exercise**: Matt has worked at galvanize 312 days, and rides the bus two ways each day.  He is thinking of buying a car, as observed that a bus is late often, 340 times to be exact.  He would like to purchase the car if the bus is *truly* late more than half the time.  Set up a z-test to decide if Matt should purchase the car.

### Two Sample Approximate Test of Population Proportions

The math in this example is a little more involved, and the Normal approximation here is much more essential than in the previous example.

#### Setup

Now that Matt is confident in his kickflips, he wants to prove that he is better at them than his friend Nick, at least in terms of consistency.

On a day of skating together, Matt attempts 80 kickflips, landing 58, before hurting his ankle.  Nick finishes the full 100, and lands 65.

#### Take a Skeptical Stance, and Clearly State This Hypothesis.

The skptical stance would be that Matt is *not* better than Nick at kickflips, he is at best, equally good.  More precisely, our null hypothesis is that the frequency Matt lands kickflips is *at most* the frequency that Nick lands kickflips.

$$ p_M = P(\text{Matt lands a kickflip}) $$
$$ p_N = P(\text{Nick lands a kickflip}) $$

$$ H_0: \ p_M \leq p_N $$

#### Create a Probablistic Model of the Stiuation Assuming the Null Hypothesis is True

Assuming that each kcikflip Matt tries is independent of all the rest (with the same for Nick), the number of kickflips Matt and Nick land in a fixed number of attempts are Binomial distributed

$$ \text{# of kickflips Matt lands} \sim Binomial(80, p_M) $$
$$ \text{# of kickflips Nick lands} \sim Binomial(100, p_N) $$

The numbers here as smaller than before, but a Normal approximation is still appropriate for each of these Binomail distributions

$$ \text{# of kickflips Matt lands} \approx Normal(80p_M, \sqrt{80p_M(1-p_M)}) $$
$$ \text{# of kickflips Nick lands} \approx Normal(100p_N, \sqrt{100p_N(1-p_N)}) $$

Now let's do something clever, **we want to compare the rates that Matt and Nick land kickflips**, so let's instead consider the **frequency** that Matt an Nick land kickflips, instead of the actual number they land.  This puts Matt and Nick on equal footing, neccessary since the attempted different numbers of kickflips

$$ \text{Frequency Matt lands kickflips in a sample} \approx Normal \left( p_M, \sqrt{\frac{p_M(1-p_M)}{80}} \right) $$
$$ \text{Frequency Nick lands kickflips in a sample} \approx Normal \left( p_N, \sqrt{\frac{p_N(1-p_N)}{100}} \right) $$

Finally, assuming that Matt landing a kickflip is independent of Nick landing a kickflip, we get a probablistic model for the **difference in sample frequecies**

$$ \text{Difference in sample frequencies between Matt and Nick} \approx Normal \left( p_M - p_N, \sqrt{\frac{p_N(1-p_N)}{100} + \frac{p_M(1-p_M)}{80}} \right) $$

**Question:** How did I calculate the mean and varaince of this final Normal distribution.

#### Bringing In the Null Hypothesis

Our null hypothesis is

$$ H_0: \ p_M \leq p_N $$
    
In the same way as before, I want to make this hypothesis maximally difficult to reject, but also have specific probabilities to work with (rather than a range).  The most conservative hypothesis is

$$ H_0: \ p_M = p_N \equiv p$$

Which makes our probabalistic model under the null reduce to

$$ \text{Difference in sample frequencies between Matt and Nick} \approx Normal \left( 0, \sqrt{\frac{p(1-p)}{100} + \frac{p(1-p)}{80}} \right) = Normal \left( 0, \sqrt{\frac{180 p(1 - p)}{8000}} \right) $$

The last problem is that we **don't know the true frequencies $p_M$ and $p_N$, nor the assumend shared fequency p**, (if we did, we wouldnt need the statistical test).  Our only resort is to substitute an estimate of the shared value from the sample

$$ \text{Difference in sample frequencies between Matt and Nick} \approx Normal \left( 0, \sqrt{\frac{180 \hat p(1- \hat p)}{8000}} \right) $$

In [ ]:
shared_sample_freq = (58.0 + 65.0) / 180
shared_sample_variance = 180 * (shared_sample_freq * (1 - shared_sample_freq)) / 8000

difference_in_proportions = stats.norm(0, np.sqrt(shared_sample_variance))

In [ ]:
fig, ax = plt.subplots(1, figsize=(16, 3))

x = np.linspace(-1, 1, num=250)
ax.plot(x, difference_in_proportions.pdf(x), linewidth=3)
ax.set_xlim(-1, 1)
ax.set_title("Distribution of Difference in Sample Frequencies Assuming $H_0$")

#### Decide how Suprised You Need to Be to Reject Your Skeptical Assumption

Tricky in this situation, depends on what Matt wants to do with the information.  

Matt wants to be a jerk, and rub it in his friends face, but to do so, he better be really sure.  So let's set the rejection threashold pretty stringently: $\alpha = 0.02$.

#### Collect Your Data

![Two People Kickflips](images/two-kickflips.jpg)

#### Calculate the Probability of Finding a Result Equally or More Extreme than Actually Observed Assuming the Probabilistic Model You Created.

The difference in proportions in the observed sample is

In [ ]:
matt_sample_freq = 58.0 / 80
nick_sample_freq = 65.0 / 100
difference_in_sample_proportions = matt_sample_freq - nick_sample_freq
print("Difference in sample proportions: {:2.2f}".format(difference_in_sample_proportions))

So the p-value for our experiment is

$$ P(\text{Difference in proportions equal to or more extreme than observed} \mid \text{Null hypothesis}) $$

In [ ]:
p_value = 1 - difference_in_proportions.cdf(difference_in_sample_proportions)
print("p-value for kickfip frequency comparison: {:2.2f}".format(p_value))

In [ ]:
fig, ax = plt.subplots(1, figsize=(16, 3))

x = np.linspace(-1, 1, num=250)
ax.plot(x, difference_in_proportions.pdf(x), linewidth=3)
ax.fill_between(x, difference_in_proportions.pdf(x), where=(x >= difference_in_sample_proportions),
                color="red", alpha=0.5)
ax.set_xlim(-1, 1)
ax.set_title("p-value Reigon")

#### Compare the p-value to Your Stated Rejection Threshold.

We're no where near confident enough in Matt's superior ability to brag, so he should back off that for the moment.

The two sample approximate test for population proportions is sometimes called the **two sample z-test**, for similar reasons as before.

**Exercise:** Matt and his friend Mike like to do integral competitions, they get out their calculus textbook, pick a random integral, then race to compute it.  Of course, the win only counts if they get the answer *correct*, which they can check with an online solver.  After 100 integrals, Matt was correct 70% of the time, and Mike was correct 77% of the time.  Is is safe to say that Mike is better at intergration than Matt?

### Two Sample Approximate Test of Population Means (Welsh's t-test)

Next, Matt and Nick want to see who can kickflip **higher**.  They set up a measuring tape, and each carefully measure the heights of 25 of thier kickflips (in feet).

In [ ]:
matt_heights = [1.5, 0.9, 0.9, 1.3, 2.1, 1.9, 1.6, 1.4, 2.3, 1.0, 1.5, 0.9, 2.1,
                1.3, 1.4, 2.1, 1.2, 1.7, 1.7, 2.0, 1.6, 1.2, 1.6, 1.7, 1.5]
nick_heights = [1.9, 1.8, 1.3, 2.3, 1.7, 1.9, 2.7, 2.1, 2.2, 1.5, 1.8, 2.0, 1.1,
                1.7, 2.1, 1.5, 2.3, 1.5, 2.0, 2.4, 1.5, 1.4, 1.6, 1.8, 1.0]

In [ ]:
fig, ax = plt.subplots(1, figsize=(16, 3))

ax.scatter(matt_heights, np.repeat(0, len(matt_heights)) + np.random.normal(0, 0.1, len(matt_heights)), s=45)
ax.scatter(nick_heights, np.repeat(1, len(nick_heights)) + np.random.normal(0, 0.1, len(matt_heights)), s=45)
ax.set_yticks([0, 1])
ax.set_yticklabels(["Matt", "Nick"])

The data looks like Nick can consistently kickflip higher than Matt, but let's construct a formal test of this.

#### Take a Skeptical Stance, and Clearly State This Hypothesis.

Skepticism here is a bit hard to interpret.  A good baseline assumption seems to be 

> there is no difference in how high Nick and Matt can kickflip.

Now there is clearly random variation in each **individual**  kickflip, so a more precise baseline assumption would be

> there is no difference in the average heights of kickflips performed by Nick and Matt.

#### Create a Probablistic Model of the Stiuation Assuming the Null Hypothesis is True

Our question concerns population averages (is Matt's population average different than Nick's).  Our measurements are **sample averages**, which, from the central limit theorem, we know are approximately normally distributed given the population average

$$ \text{Sample average of Matt's kickflips} \sim Normal \left( \mu_M, \sqrt{\frac{\sigma^2_M}{25}} \right) $$
$$ \text{Sample average of Nick's kickflips} \sim Normal \left( \mu_N, \sqrt{\frac{\sigma^2_N}{25}} \right) $$

Again, if we are willing to assume that the hight of Matt's individual kickflips are independent from Nick's, then we can compress the important information into one normal distribution

$$ \text{Difference in sample averages} \sim Normal \left( \mu_M - \mu_N, \sqrt{\frac{\sigma^2_M}{25} + \frac{\sigma^2_N}{25}} \right) $$

Under the assumption of the null hypothesis

$$ \text{Difference in sample averages} \sim Normal \left( 0, \sqrt{\frac{\sigma^2_M}{25} + \frac{\sigma^2_N}{25}} \right) $$

We're in a similar situation to last time, we do not know the population variances.

In the prior problem, our assumptions allowed us to compute the relevent varaince from the shared population frequency (which we estimated with the sample frequency).  There are two differences here

  - There are two independent varainces, which we have no reason to believe are the same.
  - The variances are independent parameters, unrelated to the mean.
  
In cases where we have to independently estiamte the variance of a normal distribution from the same samples we are testing, this estimation of the variance contributes to uncertenty in our test.  This means that the Normal distribution is then **too precise** to use as a conservative estimate of the p-value.

### Welch's t-test

To recify the problem, we first convert to a sample statistic whose variance is expected to be $1$

$$ \frac{\text{Difference in sample averages}}{\sqrt{\frac{\sigma^2_M}{25} + \frac{\sigma^2_N}{25}}} $$

Now we still have a similar issue to the two sample test of population proportions, we do not know the population varainces in the denominator of the formula, so our only recourse is to substitute in the sample variances

$$ T = \frac{\text{Difference in sample averages}}{\sqrt{\frac{\hat \sigma^2_M}{25} + \frac{\hat \sigma^2_N}{25}}} $$

In [ ]:
def welch_test_statistic(sample_1, sample_2):
    numerator = np.mean(sample_1) - np.mean(sample_2)
    denominator_sq = (np.var(sample_1) / len(sample_1)) + (np.var(sample_2) / len(sample_2))
    return numerator / np.sqrt(denominator_sq)

In [ ]:
test_statistic = welch_test_statistic(matt_heights, nick_heights)
print("Welch Test Statistic: {:2.2f}".format(test_statistic))

Unfortuantely, this changes the distribution of the test statistic.  Instead of using a normal distribution, we must not use a **Student's t-distribution**, which accounts for the extra uncertenty in estimating the two new parameters.


The t-distribution always has mean $0$ and varaince $1$, and has one parameter, the **degrees of freedom**.  Smaller degrees of freedom have heavyer tails, with the distribution becoming more normal as the degrees of freedom gets larger.

In [ ]:
dfs = [1, 2, 5, 10, 25]
x = np.linspace(-3, 3, num=250)

fig, ax = plt.subplots(1, figsize=(16, 3))

normal = stats.norm(0, 1)
ax.fill_between(x, normal.pdf(x), color="grey", alpha=0.2)
for df in dfs:
    students = stats.t(df)
    ax.plot(x, students.pdf(x), linewidth=2,
            label="Degree of Freedom: {}".format(df))
ax.legend()
ax.set_title("Student's t-distributions")


**Welch's t-test** says that in this situation, the distribution of T is well approximated with a Student's distribution with an appropriate degree of freedom.  The degrees of freedom calculation is complex, enough that it has a name, the [Welch–Satterthwaite equation](https://en.wikipedia.org/wiki/Muriel_Bristol).  The resulting application to our situation results in [Welch's t-test](https://en.wikipedia.org/wiki/Welch's_t-test).

In [ ]:
def welch_satterhwaithe_df(sample_1, sample_2):
    ss1 = len(sample_1)
    ss2 = len(sample_2)
    df = (
        ((np.var(sample_1)/ss1 + np.var(sample_2)/ss2)**(2.0)) / 
        ((np.var(sample_1)/ss1)**(2.0)/(ss1 - 1) + (np.var(sample_2)/ss2)**(2.0)/(ss2 - 1))
    )
    return df

In [ ]:
df = welch_satterhwaithe_df(nick_heights, matt_heights)
print("Degrees of Freedom for Welch's Test: {:2.2f}".format(df))

In [ ]:
x = np.linspace(-3, 3, num=250)

fig, ax = plt.subplots(1, figsize=(16, 3))
students = stats.t(df)
ax.plot(x, students.pdf(x), linewidth=2, label="Degree of Freedom: {:2.2f}".format(df))
ax.legend()
ax.set_title("Distribution of Welsh's Test Statistic Under the Null Hypothesis")

#### WARNING:

**Also, the above is not always true!**  The $T$ statistic only has a t-distribution **under the assumption that the population distributions are Normal**!  We did *not* have to assume this for *any* other test, but when we need to estimate the variance of the population, we need more structure!

If the population is very non-normal, the properties of the t-test **will fail**.  You must have some legitimate a-priori reason to believe the populations are approximately normal to use a t-test!

For this reason, many statisticians advise **AGAINST** t-tests these days, preferring non-parametric tests like the signed rank test.  **We will discuss the signed rank test this afternoon!**.

#### Decide how Suprised You Need to Be to Reject Your Skeptical Assumption

Given the bragging rights at stake here, and how much they value to friendship, Matt and Nick decide to be reasonably skeptical, so take $\alpha = 0.05$.

#### Calculate the Probability of Finding a Result Equally or More Extreme than Actually Observed Assuming the Probabilistic Model You Created.

An interesting wrinkle here.  We have no prior assumption about whether Nick is better than Matt, or Matt is better than Nick, so a difference could go in either direction.

In [ ]:
x = np.linspace(-3, 3, num=250)

fig, ax = plt.subplots(1, figsize=(16, 3))
students = stats.t(df)
ax.plot(x, students.pdf(x), linewidth=2, label="Degree of Freedom: {:2.2f}".format(df))
_ = ax.fill_between(x, students.pdf(x), where=(x >= -test_statistic), color="red", alpha=0.25)
_ = ax.fill_between(x, students.pdf(x), where=(x <= test_statistic), color="red", alpha=0.25)
ax.legend()
ax.set_title("p-value Reigon")

In [ ]:
p_value = students.cdf(test_statistic) + (1 - students.cdf(-test_statistic))
print("p-value for different average kickflip height: {:2.2f}".format(p_value))

So we have enough evidence to conclude that **one of Nick or Matt is better**.

Look's like in this case we should be able to conclude that Nick is better, let's test that.

In [ ]:
test_statistic = welch_test_statistic(nick_heights, matt_heights)

p_value = 1 - students.cdf(test_statistic)
print("p-value for Nick average kickflip height greater than Matt: {:2.3f}".format(p_value))

So Nick is clearly better.

## Non-Parametrics: Mann-Whitney Signed Rank Test

The [Mann-Whitney U-test](https://en.wikipedia.org/wiki/Mann%E2%80%93Whitney_U_test) is a modern alternative to the classical Student's and Welch's t-test that makes good use of modern computing power.  It makes **no** distributional assumptions (unlike the t-test, which assumes the populations are normal), and can always be used instead.

### Rank Sums and the Test Statistic

Let's go back to our example of Matt and Nick competing to kickflip higher.

In [ ]:
fig, ax = plt.subplots(1, figsize=(16, 3))

ax.scatter(matt_heights, np.repeat(0, len(matt_heights)) + np.random.normal(0, 0.1, len(matt_heights)), s=45)
ax.scatter(nick_heights, np.repeat(1, len(nick_heights)) + np.random.normal(0, 0.1, len(matt_heights)), s=45)
ax.set_yticks([0, 1])
ax.set_yticklabels(["Matt", "Nick"])

The idea of the Mann-Whitney test is to view this as a competition.  We let each of Nick's kicfips compete against all of Matt's kickflips, and see how many times it wins (i.e. how many of Matt's kickflips it beats).  We then add these number of wins up over all of Nick's kickflips.

In [ ]:
def count_winning_pairs(sample_1, sample_2):
    sample_1, sample_2 = np.array(sample_1), np.array(sample_2)
    n_total_wins = 0
    for x in sample_1:
        n_wins = np.sum(x > sample_2) + 0.5*np.sum(x == sample_2)
        n_total_wins += n_wins
    return n_total_wins

In [ ]:
nick_wins = count_winning_pairs(nick_heights, matt_heights)
matt_wins = count_winning_pairs(matt_heights, nick_heights)
print("Number of Nick Wins: {}".format(nick_wins))
print("Number of Matt Wins: {}".format(matt_wins))

Notice that the total number of wins is just the total number of comparisons between one of Matt's kickflips and one of Nicks

In [ ]:
print("Total Number of Wins: {}".format(nick_wins + matt_wins))
print("Total Number of Comparisons: {}".format(
    len(nick_heights)*len(matt_heights)))

The total number of wins we calculated above is called the **Mann-Whitney U Statistic**, or the **Rank Sum Statisitic**.

### The U-Test

To test the hypothesis that Nick is better than Matt, we need to adopt a Null hypothesis.  The Null for the Mann-Whitney test is directly related to which competitor is better.

> $H_0$: Matt's Kickflips are equaliy likely to be higher than Nicks as the other way around.  I.e. 
  
  $$P(\text{Height Nick Kickflip} > \text{Height Matt Kickflip}) = 0.5$$
  
As is usual, assuming this null hypothesis is true, the rank-sum statistic assums a known (but complicated) distribution.  This time we can't write down the distribution in any explicit way, but python can calculate p-values using it.

In [ ]:
res = stats.mannwhitneyu(nick_heights, matt_heights, alternative="greater")
print("p-value for Nick > Matt: {:2.3f}".format(res.pvalue))

We get the same result as with the t-test: Nick is clearly better than Matt.

## Mathematical Properties of Hypothesis Tests

The logic of hypothesis testing can seem awkward and silly when first encountered, and the great number of individual hypothesis tests you need to learn to "speak the language" can be overwhelming.  Hopefully our examples have made the idea clear and intuitive.

Let's now turn to some higher level properties of hypothesis tests.

### The Distribution of p-values Under the Null Hypothesis

Fisher gave the first definition of **p-value**, his original intention was only to devise some measure of **strength of evidence** for a scientific hypothesis.

The idea of a *rejection threashold* came later, from Neyman–Pearson.  There idea was to **control the rate of false scientific discoveries**.

To explain this, we need to study the distribution of p-values under the null hypothesis.

Suppose we are studying a question, and the null hypothesis is **actually true**.  We collect some data, and compute the p-value of this data under the null hypothesis.  The question we want to address is **how do the computed p-values behave probabalistically?**

That is, drawing a different sample will result in a different p-value, how are these p-values distributed with respect to different samples?

**Discussion:** Let's brainstorm.  Assuming the Null Hypothesis is **actually true** in some situations, what should be the distribution of p-values we observe?

The best way to do this is to construct a simulation, let's write a function that runs a (one-tailed) one sample z-test on data sampled from the null hypothesis distribution, and returns the p-value of the sampled data.

In [ ]:
def simulate_one_sample_z_test(n_simulations, sample_size=100, mu=0.0, sigma=1.0):
    normal = stats.norm(mu, sigma)
    sampling_distribution = stats.norm(mu, sigma / np.sqrt(sample_size))
    p_values = []
    for _ in range(n_simulations):
        sample = normal.rvs(sample_size)
        sample_mean = np.mean(sample)
        #sample_variance = np.var(sample)
        p_value = 1 - sampling_distribution.cdf(sample_mean)
        p_values.append(p_value)
    return p_values

Now let's run many simulations, and draw a histogram of the p-values

In [ ]:
p_values = simulate_one_sample_z_test(10**5)

fig, ax = plt.subplots(1, figsize=(16, 4))
_ = ax.hist(p_values, bins=100, alpha=0.75, density=True, color="grey")
ax.set_title("Distribution of p-values Under the Null Hypothesis")

**The p-values from a properly done hypothesis test, in the situation that the null hypothesis is true, are uniformly distributed between 0 and 1**.

### Controling the False Positive Rate

The fact above gives some insight into the idea behind the rejection threshold $\alpha$.

Recall that we set a rejection threshold *before* running the eperiment, and it is related to the weight of evidence we require before rejecting the null hypothesis.  I.e., we reject the null hypothesis when our computed p-value is less than our rejection threshold.

In the situation where the **null hypothesis is actually true**, setting a rejection threshold of, say, $0.05$ ensures that we will **only falsely reject the null hypothesis 5% of the time**.  Falsely rejecting the null hypothesis is called a **false positive**, or a **type one error**.

In [ ]:
fig, ax = plt.subplots(1, figsize=(16, 4))
_, _, patches = ax.hist(p_values, bins=100, alpha=0.75, density=True, color="grey")
for i in range(6):
    patches[i].set_color("green")
    
ax.set_title("False Positive p-values")

The green bars are the tests in our simulation that falsely reject the null hypothesis.

#### The Repeated Experiment Philosophy

> If we repeatedly and properly do a hypothesis test with rejection threshold $\alpha$ in a situation where our research hypothesis is **false**, then we will only **flasely conclude that it is true** at a rate of $\alpha$. 

Although the repeated experiment philosophy is often stated as the basis for hypothesis testing, it is actually **not** how hypothesis testing is used.  A more accurate philosophy is the

#### Worst Case Long Term False Positive Rate

> In the worst case situation that **all of our scientific hypothesis are false**, scientists using a rejection threshold of $\alpha$ for thier experiments will have, in the long term, a false positive rate of $\alpha$.

**Discussion:** Why is the second interpretation much more reasonable than the first?

**Note:** Failing to reject the null hypothesis when the scientific hypothesis is **true** is called a **false negative** or a **type two error**.

Studying the type two error rate takes some new concepts (mainly statistical power), which will be the subject of tomorrow's lesson.

### Multiple Testing: The Problem

Suppose I am considering writing a paper for the **American Journal of Skateboarding Research**.  I have collected data about kickflips in various situations, different terrain, different weather, different shoes, different warmup routines.  I have various questions I would like to ask of this data:

  - Does using a crisp, new board increase the height of kickflips?
  - Do dry, sunny days increase the height of kickflips?
  - Do broken in (as opposed to new) shoes increase the hieght of kickflips?
  - Does an aerobic warmup routine increase the height of kickflips?
  
Suppose that 

  - All my alternative hypothesies are, in reality, false. 
  - I plan to test all these hypothesies using my collected data, and write a paper in **AJSR** if I get a positive result for any of them.
  
**Question:** If I preform each of these four seperate tests at a threshold of $0.05$, what is the true rate at which I reject **at least one** of these hypothesies and publish a false paper in **AJSR**?

The calculation here is a standard probabalistic arguemnt.

  - The rate I falsely reject a single hypothesis is $\alpha$.
  - The rate I do **not** falsely reject a single hypothesis is $1 - \alpha$.
  - The rate I do **not** falsely reject **each and every one** of the hypothesies is $(1 - \alpha)^4$.
  - The rate I **do** falsely reject **at least one** of the hypothesies is $1 - (1 - \alpha)^4$.

In [ ]:
combined_false_positive_rate = 1 - (1 - 0.05)**4

print("True combined false positive rate: {:2.2f}".format(combined_false_positive_rate))

Let's look at this another way.  Suppose that the four p-values we get are $p_1, p_2$, $p_3$, and $p_4$.  Then we falsely reject the combined hypothesis whenever at least one of the p-values is less than $0.05$

$$ min(p_1, p_2, p_3, p_4) < 0.05 $$

To keep the false positive rate under control, we would need this to happen only $5\%$ of the time.

We can re-do our simulation and see how often a composite of three hypothesies is falsely rejected:

In [ ]:
p_values = simulate_one_sample_z_test(4*10**5)
p_values_combined = np.array([min(w, x, y, z) for w, x, y, z in 
                              zip(p_values[::4], p_values[1::4], p_values[2::4], p_values[3::4])])

fig, ax = plt.subplots(1, figsize=(16, 4))
_, _, patches = ax.hist(p_values_combined, bins=100, alpha=0.75, density=True, color="grey")
for i in range(6):
    patches[i].set_color("green")
    
ax.set_title("False Positive p-values")

In [ ]:
combined_false_positive_rate = np.sum(p_values_combined <= 0.05) / float(len(p_values_combined))
print("Combined False Positive Rate: {:2.2f}".format(combined_false_positive_rate))

So about 18% of the time, we will end up publishing a false paper.  This is **BAD**.

### Bonferroni Correction

The **Bonferroni Correction** is a popular way to rectify the over testing issue.

Suppose we want to test a combined hypothesis as a threshold of $\alpha$.  The bonferroni correction procedure then tests each of the individual hypothesies at a threshold of

$$ \alpha_\text{Bonferroni} = \frac{\alpha}{\text{# of Hypothesies in Combined Hypothesis}} $$

Let's check that this fixes the issue in our simulation.

In [ ]:
combined_false_positive_rate_corrected = np.sum(
    p_values_combined <= 0.05 / 4) / float(len(p_values_combined))
print("Combined False Positive Rate: {:2.2f}".format(combined_false_positive_rate_corrected))

### Mutiple Testing Over Time: Discussion

You're studying the impact of changing the layout of your skateboarding website.  The website is intended to map out the skating spots in cities around the country, user's can add spots, photos, and comments.

You've made some changes to the comment system, and are hoping that it will draw more forum comments from users.

Your plan to test this is to split users of your site into two groups, one group will always see the new layout, and one will always see the old.  You plan to run the site this way for two months, and in the end test whether the users with the new layouts generated more forum comments than those under the old layouts.

Let's simulate some data under the **truth** that the new website is **slightly worse**.

In [ ]:
np.random.seed(132)

old_forum_comment_rate = 0.2   # Old Website: Better.
new_forum_comment_rate = 0.18  # New Website: Slightly worse.
commenters_per_day = stats.poisson(2)

# Same Number of Commenters for Each
commenters_per_day_old = commenters_per_day.rvs(2*31)
commenters_per_day_new = commenters_per_day.rvs(2*31)

# 
comments_per_day_old = [
    stats.binom(commenters, old_forum_comment_rate).rvs(1)
    for commenters in commenters_per_day_old]
comments_per_day_new = [
    stats.binom(commenters, new_forum_comment_rate).rvs(1)
    for commenters in commenters_per_day_new]

cumlative_comments_per_day_old = np.cumsum(comments_per_day_old)
cumlative_comments_per_day_new = np.cumsum(comments_per_day_new)

In [ ]:
fig, ax = plt.subplots(1, figsize=(16, 4))

x = np.arange(1, 2*31 + 1)
ax.plot(x, cumlative_comments_per_day_old, label="Old Layout")
ax.plot(x, cumlative_comments_per_day_new, label="New Layout")
ax.legend()
ax.set_xlabel("Day")
ax.set_ylabel("Cumulative Number of Comments")

ax.set_title("Cumulative Number of Comments over Two Months")

The data we collected makes it look like the new website is **better**.

In [ ]:
rate_of_comments_new_layout = np.sum(comments_per_day_new) / float(np.sum(commenters_per_day_new))
rate_of_comments_old_layout = np.sum(comments_per_day_old) / float(np.sum(commenters_per_day_old))

print("Rate of comments, new layout: {:2.2f}".format(rate_of_comments_new_layout))
print("Rate of comments, old layout: {:2.2f}".format(rate_of_comments_old_layout))

But our testing framework tells us the truth, we do not reject the null that the new website is not as good as the old.

In [ ]:
def two_sample_test_of_population_proportions(commentators_new, comments_new, commentators_old, comments_old):
    difference_in_sample_proportions = (comments_new / float(commentators_new)) - (comments_old / float(commentators_old))
    overall_proportion = (comments_new + comments_old) / float(commentators_new + commentators_old)
    test_varaince = ((comments_new + comments_old) * overall_proportion * (1 - overall_proportion)) / (commentators_new + commentators_old)
    test_distribution = stats.norm(0, np.sqrt(test_varaince))
    p_value = 1 - test_distribution.cdf(difference_in_sample_proportions)
    return p_value

In [ ]:
p_value = two_sample_test_of_population_proportions(
    np.sum(commenters_per_day_new), np.sum(comments_per_day_new), 
    np.sum(commenters_per_day_old), np.sum(comments_per_day_old)
)

print("p-value for full experiment: {:2.2f}".format(p_value))

Suppose that we cheated.  We looked at the data every day, and ran a hypothesis test on that days data, stopping if we ever got a significant result.

Now let's see the sequence of p-values we would get if we run the test every day.

In [ ]:
cumlative_comments_per_day_old = np.cumsum(comments_per_day_old)
cumlative_comments_per_day_new = np.cumsum(comments_per_day_new)
cumlative_commenters_per_day_old = np.cumsum(commenters_per_day_old)
cumlative_commenters_per_day_new = np.cumsum(commenters_per_day_new)

p_values = [
    two_sample_test_of_population_proportions(
        cumlative_commenters_per_day_new[i], 
        cumlative_comments_per_day_new[i],
        cumlative_commenters_per_day_old[i], 
        cumlative_comments_per_day_old[i]
    )
    for i in range(1, 2*31)
]

In [ ]:
fig, ax = plt.subplots(1, figsize=(16, 3))

ax.plot(range(1, 2*31), p_values)
ax.set_title("P-values Over Time")
ax.set_xlabel("Day")
ax.set_ylabel("p-value")

If we peeked at our data any time between the 10'th and 16'th day, we would have made the wrong conclusion!

In [ ]:
fig, ax = plt.subplots(1, figsize=(16, 3))

ax.plot(range(1, 2*31), p_values)
ax.axvline(10, linestyle="--")
ax.axvline(16, linestyle="--")
ax.axvspan(10, 16, alpha=0.25, color='red')
ax.set_title("P-values Over Time")
ax.set_xlabel("Day")
ax.set_ylabel("p-value")

**Discussion:** What is the danger in testing every day, and stopping the experiment if a signifigant result is found?  Everything seemed to go ok here!

# Extra!

## Appendix: Chi Squared Tests

### Example: Linear Congruential Generators

You've been contracted to create a small game that will run on an [embedded system](https://en.wikipedia.org/wiki/Embedded_system) (think, for example, a [Tamagotchi](https://en.wikipedia.org/wiki/Tamagotchi) ).  The system does not have many resources, so you have to create your own random number generator.

After some research, you hit on a lightweight solution, a [linear congruential generator](https://en.wikipedia.org/wiki/Linear_congruential_generator).  To scope things out, you code a simple generator in python.

In [ ]:
class LinearCongruentialGenerator:
    
    def __init__(self, a, c, modulus, seed):
        self._a = a
        self._c = c
        self._modulus = modulus
        self._seed = seed
        
    def next(self):
        next_sample = (self._a * self._seed + self._c) % self._modulus
        self._seed = next_sample
        return next_sample
    
    def sample(self, n):
        L = []
        for _ in range(n):
            L.append(lcm.next())
        return np.array(L)

And lookup some parameters to use online.

In [ ]:
lcm = LinearCongruentialGenerator(48271, 0, 2**31 - 1, 1)

You want to test if you're code really works, which involves assessing the "randomness" of your generator.  

**Discussion:** How would you assess the quality of your random numbers.

Your intention in the app is to use the generator to create random six sided dice rolls, so you plan to test whether the generator is appropriate for that application.

You have a strong belief that the die is fair, as this code is based on an industry standard method, so your goal here is to detect whether the random number generator is *unfair.*

The app will only be used in small bursts of activity, so you decide to run these tests on a smallish sample.

In [ ]:
dice_rolls = lcm.sample(50) % 6
dice_freqs = np.bincount(dice_rolls)

fig, ax = plt.subplots(1, figsize=(16, 4))
ax.bar(range(1, 7), dice_freqs, align='center')

### The Chi-Squared Test for Distribution

The first step is to arrange our results into a **contingency table**, which compares the expected frequency to the observed frequency

In [ ]:
import pandas as pd
dice_rolls_contingency_table = pd.DataFrame(
    {'expected': np.repeat(8.3, 6) , 'actual': dice_freqs}
)
print(dice_rolls_contingency_table)

The **Chi Squared Test for Distribution**, (also known as the **Chi Squared Test for Goodness of Fit** tests the hypothesis that the **actual** data was not generated from a discrete distributition which has expected frequencies as calcualted.

>$H_0$: The data was generated from a dicrete distribution with the given expected frequencies.

> $H_{a}$: The data was *not* generated from a discrete distribution with the given expected frequencies.

In out situation, a shorter way to express this is

>$H_0$: The data generated from the random number generator is consistent with a fair die.

> $H_{a}$: The data generated from the random number generator is not consistent with a fair die.

**Note** a point of awkwardness here.  We would like to conclude the die is fair, so really our antagonistic hypothesis should be that the die is unfair.  But assuming the die is unfair does not let us create a probabilistic model for the situation under the null.

Unforunately, this is the common logic of Chi squared tests for distribution.

#### Assemble the Contingency Table

We've already done this step.  Good to go.

#### Compute the Test Statistic

The first step in the Chi Squared test is to compute the following test statistic:

$$ T = \sum_i \frac{(O_i - E_i)^2}{E_i} $$

In [ ]:
def chi_squared_test_statistic(observeds, expecteds):
    numerators = (observeds - expecteds)**2
    ratios = numerators / expecteds
    return np.sum(ratios)

The logic of the Chi Squared test is that this quantity follows a certain distribution, the chi squared distribution.

#### Preform the Test

The Chi Squared distribution has one parameter, the degrees of freedom.

In [ ]:
dfs = [1, 2, 3, 4, 5]
x = np.linspace(0, 3, num=250)

fig, ax = plt.subplots(1, figsize=(16, 3))

for df in dfs:
    chisq = stats.chi2(df)
    ax.plot(x, chisq.pdf(x), linewidth=2,
            label="Degree of Freedom: {}".format(df))
ax.set_ylim(0, 0.5)
ax.legend()

The Chi squared test provides the degrees of freedom as:
    
$$ \text{Number of Cells in the Contingency Table} - \text{Number of Parameters in Distribution} + 1$$

So let's apply this to our contingency tables

#### The Dice Rolls

In this case, our degrees of freedom for the Chi Squared distribution is

$$ 6 - 1 + 1 = 6 $$

And the test statistic is

In [ ]:
T = chi_squared_test_statistic(dice_rolls_contingency_table.actual, 
                               dice_rolls_contingency_table.expected)

So our p-value for the test is

In [ ]:
test_distribution = stats.chi2(6)
p_value = 1 - test_distribution.cdf(T)
print("p-value for dice rolls: {:2.2f}".format(p_value))

Looks like this experiment does *not* give us enough evidence to conclude that our random number generator produces fair rolls.  Looks like we should do more research.

**Exercise:** Do the other test, of consecutive differences.

## Appendix: Secret Code to Find Random Seed that Rejects In Forum Example

In [ ]:
def reject_on_day(seed, burn_in=0):
    np.random.seed(seed)

    old_forum_comment_rate = 0.2   # Old Website: Better.
    new_forum_comment_rate = 0.18  # New Website: Slightly worse.
    commenters_per_day = stats.poisson(2)

    # Same Number of Commenters for Each
    commenters_per_day_old = commenters_per_day.rvs(2*31)
    commenters_per_day_new = commenters_per_day.rvs(2*31)

    # 
    comments_per_day_old = [
        stats.binom(commenters, old_forum_comment_rate).rvs(1)
        for commenters in commenters_per_day_old]
    comments_per_day_new = [
        stats.binom(commenters, new_forum_comment_rate).rvs(1)
        for commenters in commenters_per_day_new]

    cumlative_comments_per_day_old = np.cumsum(comments_per_day_old)
    cumlative_comments_per_day_new = np.cumsum(comments_per_day_new)

    p_values = np.array([
        two_sample_test_of_population_proportions(
            cumlative_commenters_per_day_new[i], cumlative_comments_per_day_new[i],
            cumlative_commenters_per_day_old[i], cumlative_comments_per_day_old[i]
        )
        for i in range(1, 2*31)
    ])
    
    return any(p_values[burn_in:] <= 0.05)

In [ ]:
for seed in range(1, 50000):
    if reject_on_day(seed, burn_in=15):
        break

In [ ]:
seed

In [ ]:
import numpy as np
np.random.seed(222)

In [ ]:
np.random.uniform(0, 1, size=50)

In [ ]:
np.random.seed(222)
np.random.uniform(0, 1, size=50)